# 201: Exampville Mode Choice

Welcome to Exampville, the best simulated town in this here part of the internet!

Exampville is a demonstration provided with Larch that walks through some of the 
data and tools that a transportation planner might use when building a travel model. 

In [1]:
# HIDDEN
import larch.numba as lx
import jax.numpy as jnp
from pytest import approx

/Users/jeffnewman/LocalGit/larch/larch/numba/model.py:23: UserWarning: 

### larch.numba is experimental, and not feature-complete ###
 the first time you import on a new system, this package will
 compile optimized binaries for your machine, which may take 
 a little while, please be patient 

  warnings.warn( ### EXPERIMENTAL ### )
OMP: Info #273: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [2]:
import os
import numpy as np
import pandas as pd 
import larch.numba as lx
from larch import P, X

In this example notebook, we will walk through the creation of a tour mode choice model.

To begin, we'll re-load the tours and skims data from the 
data setup example.

In [3]:
hh, pp, tour, skims = lx.example(200, ['hh', 'pp', 'tour', 'skims'])

## Preprocessing

The Exampville data output contains a set of files similar to what we might
find for a real travel survey: network skims, and tables of households, persons,
and tours.  We'll need to connect these tables together to create a composite dataset
for mode choice model estimation, using the DataTree structure.

In [4]:
from addicty import Dict

Mode = Dict(
    DA = 1,
    SR = 2,
    Walk = 3,
    Bike = 4,
    Transit = 5,
).freeze()

In [5]:
tour_dataset = lx.Dataset.from_idco(tour.set_index('TOURID'), alts=Mode)
od_skims = lx.Dataset.from_omx(skims)

dt = lx.DataTree(
    tour=tour_dataset,
    hh=hh.set_index('HHID'),
    person=pp.set_index('PERSONID'),
    od=od_skims,
    do=od_skims,
    relationships=(
        "tours.HHID @ hh.HHID",
        "tours.PERSONID @ person.PERSONID",
        "hh.HOMETAZ @ od.otaz",
        "tours.DTAZ @ od.dtaz",
        "hh.HOMETAZ @ do.dtaz",
        "tours.DTAZ @ do.otaz",
    ),
)

In Exampville, there are only two kinds of trips: 

- work (purpose=1) and 
- non-work (purpose=2). 

We want to estimate a mode choice model for work trips, 
so we’ll begin by excluding all the other trips:

In [6]:
dt_work = dt.query_cases("TOURPURP == 1")

## Model Definition

And then we are ready to create our model.

In [7]:
m = lx.Model(datatree = dt_work)
m.title = "Exampville Work Tour Mode Choice v1"

We will explicitly define the set of utility functions 
we want to use.  Because the DataFrames we are using to 
serve data to this model contains exclusively `idco` format
data, we'll use only the `utility_co` mapping to define
a unique utility function for each alternative.

In [8]:
m.utility_co[Mode.DA] = (
        + P.InVehTime * X.AUTO_TIME
        + P.Cost * X.AUTO_COST # dollars per mile
)

m.utility_co[Mode.SR] = (
        + P.ASC_SR
        + P.InVehTime * X.AUTO_TIME
        + P.Cost * (X.AUTO_COST * 0.5) # dollars per mile, half share
        + P("LogIncome:SR") * X("log(INCOME)")
)

m.utility_co[Mode.Walk] = (
        + P.ASC_Walk
        + P.NonMotorTime * X.WALK_TIME
        + P("LogIncome:Walk") * X("log(INCOME)")
)

m.utility_co[Mode.Bike] = (
        + P.ASC_Bike
        + P.NonMotorTime * X.BIKE_TIME
        + P("LogIncome:Bike") * X("log(INCOME)")
)

m.utility_co[Mode.Transit] = (
        + P.ASC_Transit
        + P.InVehTime * X.TRANSIT_IVTT
        + P.OutVehTime * X.TRANSIT_OVTT
        + P.Cost * X.TRANSIT_FARE
        + P("LogIncome:Transit") * X('log(INCOME)')
)

To write a nested logit mode, we'll attach some nesting nodes to the 
model's `graph`.  Each `new_node` allows us to define the set of 
codes for the child nodes (elemental alternatives, or lower level nests)
as well as giving the new nest a name and assigning a logsum parameter.
The return value of this method is the node code for the newly created 
nest, which then can potenially be used as a child code when creating
a higher level nest.  We do this below, adding the 'Car' nest into the 
'Motor' nest.

In [9]:
Car = m.graph.new_node(parameter='Mu:Car', children=[Mode.DA, Mode.SR], name='Car')
NonMotor = m.graph.new_node(parameter='Mu:NonMotor', children=[Mode.Walk, Mode.Bike], name='NonMotor')
Motor = m.graph.new_node(parameter='Mu:Motor', children=[Car, Mode.Transit], name='Motor')

Let's visually check on the nesting structure.

In [10]:
m.graph

The tour mode choice model's choice variable is indicated by 
the code value in 'TOURMODE', and this can be 
defined for the model using `choice_co_code`.

In [11]:
m.choice_co_code = 'TOURMODE'

We can also give a dictionary of availability conditions based 
on values in the `idco` data, using the `availability_co_vars`
attribute.  Alternatives that are always available can be indicated
by setting the criterion to 1.

In [12]:
m.availability_co_vars = {
    Mode.DA: 'AGE >= 16',
    Mode.SR: 1,
    Mode.Walk: 'WALK_TIME < 60',
    Mode.Bike: 'BIKE_TIME < 60',
    Mode.Transit: 'TRANSIT_FARE>0',
}

Then let's prepare this data for estimation.  Even though the
data is already in memory, the `load_data` method is used to 
pre-process the data, extracting the required values, pre-computing 
the values of fixed expressions, and assembling the results into
contiguous arrays suitable for computing the log likelihood values
efficiently.

## Model Estimation

We can check on some important statistics of this loaded data even
before we estimate the model.

In [13]:
m.choice_avail_summary()

,name,chosen,available,availability condition
1,DA,6052,7564,AGE >= 16
2,SR,810,7564,1
3,Walk,196,4179,WALK_TIME < 60
4,Bike,72,7564,BIKE_TIME < 60
5,Transit,434,4199,TRANSIT_FARE>0
6,Car,6862,7564,
7,NonMotor,268,7564,
8,Motor,7296,7564,
< Total All Alternatives >,,7564,,


In [14]:
# TEST
summary = m.choice_avail_summary()
assert (summary.to_markdown()) == '''
|                            | name     |   chosen | available   | availability condition   |
|:---------------------------|:---------|---------:|:------------|:-------------------------|
| 1                          | DA       |     6052 | 7564        | AGE >= 16                |
| 2                          | SR       |      810 | 7564        | 1                        |
| 3                          | Walk     |      196 | 4179        | WALK_TIME < 60           |
| 4                          | Bike     |       72 | 7564        | BIKE_TIME < 60           |
| 5                          | Transit  |      434 | 4199        | TRANSIT_FARE>0           |
| 6                          | Car      |     6862 | 7564        |                          |
| 7                          | NonMotor |      268 | 7564        |                          |
| 8                          | Motor    |     7296 | 7564        |                          |
| < Total All Alternatives > |          |     7564 |             |                          |
'''[1:-1]

If we are satisfied with the statistics we see above, we
can go ahead and estimate the model.

In [15]:
m.set_cap(20) # improves optimization stability
result = m.maximize_loglike()

,value,initvalue,nullvalue,minimum,maximum,holdfast,note,best
ASC_Bike,-0.258018,0.0,0.0,-20.000,20.0,0,,-0.258018
ASC_SR,1.422912,0.0,0.0,-20.000,20.0,0,,1.422912
ASC_Transit,6.754512,0.0,0.0,-20.000,20.0,0,,6.754512
ASC_Walk,8.621509,0.0,0.0,-20.000,20.0,0,,8.621509
Cost,-0.175693,0.0,0.0,-20.000,20.0,0,,-0.175693
InVehTime,-0.123722,0.0,0.0,-20.000,20.0,0,,-0.123722
LogIncome:Bike,-0.196983,0.0,0.0,-20.000,20.0,0,,-0.196983
LogIncome:SR,-0.193812,0.0,0.0,-20.000,20.0,0,,-0.193812
LogIncome:Transit,-0.557151,0.0,0.0,-20.000,20.0,0,,-0.557151
LogIncome:Walk,-0.522787,0.0,0.0,-20.000,20.0,0,,-0.522787


In [16]:
# TEST
assert result.loglike == approx(-3493.0397298749467)

After we find the best fitting parameters, we can compute
some variance-covariance statistics, incuding standard errors of
the estimates and t statistics, using `calculate_parameter_covariance`.

In [17]:
m.calculate_parameter_covariance()

Then we can review the results in a variety of report tables.

In [18]:
m.parameter_summary()

,Value,Std Err,t Stat,Signif,Null Value
ASC_Bike,-0.258,1.34,-0.19,,0.00
ASC_SR,1.42,1.00,1.42,,0.00
ASC_Transit,6.75,2.06,3.27,**,0.00
ASC_Walk,8.62,1.14,7.57,***,0.00
Cost,-0.176,0.120,-1.47,,0.00
InVehTime,-0.124,0.0292,-4.24,***,0.00
LogIncome:Bike,-0.197,0.124,-1.59,,0.00
LogIncome:SR,-0.194,0.135,-1.43,,0.00
LogIncome:Transit,-0.557,0.169,-3.29,***,0.00
LogIncome:Walk,-0.523,0.100,-5.21,***,0.00


In [19]:
# TEST
assert (m.parameter_summary().data.to_markdown()) == '''
|                   |   Value |   Std Err |   t Stat | Signif   |   Null Value |
|:------------------|--------:|----------:|---------:|:---------|-------------:|
| ASC_Bike          |  -0.258 |    1.34   |    -0.19 |          |            0 |
| ASC_SR            |   1.42  |    1      |     1.42 |          |            0 |
| ASC_Transit       |   6.75  |    2.06   |     3.27 | **       |            0 |
| ASC_Walk          |   8.62  |    1.14   |     7.57 | ***      |            0 |
| Cost              |  -0.176 |    0.12   |    -1.47 |          |            0 |
| InVehTime         |  -0.124 |    0.0292 |    -4.24 | ***      |            0 |
| LogIncome:Bike    |  -0.197 |    0.124  |    -1.59 |          |            0 |
| LogIncome:SR      |  -0.194 |    0.135  |    -1.43 |          |            0 |
| LogIncome:Transit |  -0.557 |    0.169  |    -3.29 | ***      |            0 |
| LogIncome:Walk    |  -0.523 |    0.1    |    -5.21 | ***      |            0 |
| Mu:Car            |   0.259 |    0.181  |    -4.1  | ***      |            1 |
| Mu:Motor          |   0.802 |    0.201  |    -0.99 |          |            1 |
| Mu:NonMotor       |   0.854 |    0.112  |    -1.3  |          |            1 |
| NonMotorTime      |  -0.266 |    0.0163 |   -16.29 | ***      |            0 |
| OutVehTime        |  -0.255 |    0.0646 |    -3.95 | ***      |            0 |
'''[1:-1]

In [20]:
m.estimation_statistics()

<xmle.Elem 'div' with 1 children>

In [26]:
m.dataset

<larch.Dataset>
Dimensions:    (TOURID: 7564, _altid_: 5, var_co: 8)
Coordinates:
  * TOURID     (TOURID) int64 0 1 3 7 10 13 ... 20728 20729 20731 20734 20736
    alt_names  (_altid_) <U7 'DA' 'SR' 'Walk' 'Bike' 'Transit'
  * _altid_    (_altid_) int64 1 2 3 4 5
  * var_co     (var_co) <U12 'AUTO_COST' 'AUTO_TIME' ... 'log(INCOME)'
Data variables:
    co         (TOURID, var_co) float64 0.5885 5.044 8.407 ... 18.02 34.77 10.05
    ch         (TOURID, _altid_) float64 1.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    av         (TOURID, _altid_) int8 1 1 1 1 0 1 1 0 1 1 ... 1 0 1 1 1 1 1 1 1
Attributes:
    _caseid_:  TOURID
    _altid_:   _altid_

In [33]:
m.dataset

<larch.Dataset>
Dimensions:    (TOURID: 7564, _altid_: 5, var_co: 8)
Coordinates:
  * TOURID     (TOURID) int64 0 1 3 7 10 13 ... 20728 20729 20731 20734 20736
    alt_names  (_altid_) <U7 'DA' 'SR' 'Walk' 'Bike' 'Transit'
  * _altid_    (_altid_) int64 1 2 3 4 5
  * var_co     (var_co) <U12 'AUTO_COST' 'AUTO_TIME' ... 'log(INCOME)'
Data variables:
    co         (TOURID, var_co) float64 0.5885 5.044 8.407 ... 18.02 34.77 10.05
    ch         (TOURID, _altid_) float64 1.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    av         (TOURID, _altid_) int8 1 1 1 1 0 1 1 0 1 1 ... 1 0 1 1 1 1 1 1 1
Attributes:
    _caseid_:  TOURID
    _altid_:   _altid_

In [38]:
av = jnp.asarray(m.dataset['av'])
av

DeviceArray([[1, 1, 1, 1, 0],
             [1, 1, 0, 1, 1],
             [1, 1, 1, 1, 0],
             [1, 1, 1, 1, 0],
             [1, 1, 1, 1, 0],
             ...,
             [1, 1, 1, 1, 1],
             [1, 1, 0, 1, 1],
             [1, 1, 0, 1, 1],
             [1, 1, 0, 1, 1],
             [1, 1, 1, 1, 1]], dtype=int8)

In [39]:
co = jnp.asarray(m.dataset['co'])
ch = jnp.asarray(m.dataset['ch'])
av = jnp.asarray(m.dataset['av'])
